In [3]:
import pandas as pd
import numpy as np

In [5]:
DATABASE_ADDRESS = "../Data/procedureevents_small.csv"
CHUNK_SIZE = 500

In [10]:
marks = {
    220045 : "Heart Rate", \
    220210 : "Respiratory Rate", \
    220179 : "Non Invasive Blood Pressure systolic", \
    220180 : "Non Invasive Blood Pressure diastolic", \
    220277 : "O2 saturation pulseoxymetry", \
    223762 : "Temperature Celsius", \
    # 223761 : "Temperature Fahrenheit"
}

# 建立一個CLASS容器來裝每一個PATIENT
class Patient(dict):
    def __init__(self, subject_id=None, hadm_id=None, charttime=None):
        self.update({
            "subject_id":   subject_id, \
            "hadm_id":      hadm_id, \
            "charttime":    charttime
        })
        for label_ in marks:
            self[label_] = None

    def __eq__(self, other):
        return self["hadm_id"] == other["hadm_id"] and self["charttime"] == other["charttime"]
    
    def __add__(self, other):
        for i in other:
            if other[i] != None:
                self[i] = other[i]
        return self
    
    def add(self, value, id_):
        if id_ == 223761:
            # Fahrenheit -> Celsius
            id_ = 223762
            value = round((float(value) - 32) * 5 / 9, 1)
        
        if id_ not in marks:
            return self
            
        self[id_] = value
        return self
    
    def allna(self, ):
        return all([self[i]==None for i in marks])

In [19]:
reader = pd.read_csv(DATABASE_ADDRESS, encoding="unicode_escape" , iterator=True, chunksize=CHUNK_SIZE)

hold = Patient()

cnt = 0
for cnt, df in enumerate(reader):
    df['charttime'] = pd.to_datetime(df['charttime']).round("10min")
    for idx, row in df.iterrows():
        temp = Patient(row["subject_id"], row["hadm_id"], row["charttime"]).add(value=row["value"], id_=row["itemid"])
        
        if hold == temp:
            hold += temp
        else:
            if not hold.allna():
                cnt += 1
                
            hold = temp
        
    break
cnt

18

In [21]:

df = pd.read_csv(DATABASE_ADDRESS, usecols=["subject_id", "hadm_id", "charttime", "itemid", "value"], encoding="unicode_escape")
df = df[df['itemid'].isin(marks)]
wide_table = df.pivot_table(index=["subject_id", "hadm_id", "charttime"], columns='itemid',values='value').reset_index() 

/var/folders/yh/tsprmwzj5195mqsvn9sq79380000gn/T/ipykernel_4410/1191566363.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATABASE_ADDRESS, usecols=["subject_id", "hadm_id", "charttime", "itemid", "value"], encoding="unicode_escape")


In [22]:
wide_table

itemid,subject_id,hadm_id,charttime,220045,220179,220180,220210,220277,223762
0,10003700,28623837,2165/4/24 05:28,NaN,152.0,97.0,NaN,NaN,NaN
1,10003700,28623837,2165/4/24 05:30,65.0,NaN,NaN,14.0,NaN,NaN
2,10003700,28623837,2165/4/24 05:31,NaN,NaN,NaN,NaN,100.0,NaN
3,10003700,28623837,2165/4/24 06:00,56.0,126.0,78.0,14.0,100.0,NaN
4,10003700,28623837,2165/4/24 06:09,55.0,NaN,NaN,14.0,100.0,NaN
...,...,...,...,...,...,...,...,...,...
102923,10903224,26582922,2184/3/31 19:54,NaN,NaN,NaN,NaN,100.0,NaN
102924,10903224,26582922,2184/3/31 20:00,73.0,137.0,82.0,21.0,100.0,NaN
102925,10903224,26582922,2184/3/31 20:02,68.0,NaN,NaN,16.0,100.0,NaN
102926,10903224,26582922,2184/3/31 21:00,61.0,109.0,68.0,15.0,100.0,NaN


In [25]:
wide_table.dropna(how='all', subset=[i for i in marks])

itemid,subject_id,hadm_id,charttime,220045,220179,220180,220210,220277,223762
0,10003700,28623837,2165/4/24 05:28,NaN,152.0,97.0,NaN,NaN,NaN
1,10003700,28623837,2165/4/24 05:30,65.0,NaN,NaN,14.0,NaN,NaN
2,10003700,28623837,2165/4/24 05:31,NaN,NaN,NaN,NaN,100.0,NaN
3,10003700,28623837,2165/4/24 06:00,56.0,126.0,78.0,14.0,100.0,NaN
4,10003700,28623837,2165/4/24 06:09,55.0,NaN,NaN,14.0,100.0,NaN
...,...,...,...,...,...,...,...,...,...
102923,10903224,26582922,2184/3/31 19:54,NaN,NaN,NaN,NaN,100.0,NaN
102924,10903224,26582922,2184/3/31 20:00,73.0,137.0,82.0,21.0,100.0,NaN
102925,10903224,26582922,2184/3/31 20:02,68.0,NaN,NaN,16.0,100.0,NaN
102926,10903224,26582922,2184/3/31 21:00,61.0,109.0,68.0,15.0,100.0,NaN


In [19]:
A = [i for i in marks]